<a href="https://colab.research.google.com/github/simon-clematide/colab-notebooks-for-teaching/blob/main/g2p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# g2p with Neural Transducer
This notebook allows the training and evaluation of  g2p models for any language. It uses the sigmorphon tsv-separated input format.

## Setup
The most important step is to use a `T4 GPU` for efficientt batch training with larger datasets and larger models!
Please select now Runtime>Change Runtime Setting!

After that install the neural transducer package. If you change the runtime afterwards all local data will be lost.

In [7]:
! pip install  git+https://github.com/simon-clematide/il-reimplementation.git@development-for-colab#egg=neural-transducer

  Cloning https://github.com/simon-clematide/il-reimplementation.git (to revision development-for-colab) to /tmp/pip-install-xk4yl6f0/neural-transducer_94c36a2a9e6f44189023fd701dcb694b
  Running command git clone --filter=blob:none --quiet https://github.com/simon-clematide/il-reimplementation.git /tmp/pip-install-xk4yl6f0/neural-transducer_94c36a2a9e6f44189023fd701dcb694b
  Running command git checkout -b development-for-colab --track origin/development-for-colab
  Switched to a new branch 'development-for-colab'
  Branch 'development-for-colab' set up to track remote branch 'development-for-colab' from 'origin'.
  Resolved https://github.com/simon-clematide/il-reimplementation.git to commit 67f4ba213bfbc4c607100d67edb2d9893ea02977
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neural-transducer: filename=neural_transducer-0.2.1-py3-none-any.whl size=35819 sha256=dc885e19a502319

This installs the main command line commands for training, evaluation and prediction: `trans-train`

See the commandline help for more information.

In [2]:
! trans-train -h

usage: trans-train [-h] [--pytorch-seed PYTORCH_SEED] [--train TRAIN]
                   [--precomputed-train PRECOMPUTED_TRAIN] [--save-precomputed-train]
                   [--vocabulary VOCABULARY] --dev DEV [--test TEST] --output OUTPUT [--nfd]
                   [--char-dim CHAR_DIM] [--feat-dim FEAT_DIM] [--action-dim ACTION_DIM]
                   [--enc-type {lstm,transformer}] [--dec-hidden-dim DEC_HIDDEN_DIM]
                   [--dec-layers DEC_LAYERS] [--beam-width BEAM_WIDTH] [--patience PATIENCE]
                   [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--eval-batch-size EVAL_BATCH_SIZE]
                   [--loss-reduction {sum,mean}] [--grad-accumulation GRAD_ACCUMULATION]
                   [--train-subset-eval-size TRAIN_SUBSET_EVAL_SIZE]
                   [--optimizer {adam,adamw,adadelta}] [--scheduler {inv_sr,reduce_on_plateau}]
                   [--sed-em-iterations SED_EM_ITERATIONS] [--sed-params SED_PARAMS]
                   [--device DEVICE]

Train a 

In [3]:
! ls /usr/local/lib/python3.10/dist-packages/trans/

actions.py     grid_search.py	  optimal_expert_substitutions.py  sed.py	  utils.py
encoders.py    __init__.py	  optimizers.py			   train.py	  vocabulary.py
ensembling.py  optimal_expert.py  __pycache__			   transducer.py


## Load sigmorphon data sets
The sigmorphon 2021 contains low-resource training setup (800 training items per language), medium (8000 training items), and one buggy high-resource (33344 training items).

In [2]:
! git clone --depth 1 https://github.com/sigmorphon/2021-task1.git

Cloning into '2021-task1'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 97 (delta 2), reused 71 (delta 0), pack-reused 0
Receiving objects: 100% (97/97), 1.28 MiB | 10.36 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [3]:
! ln -s 2021-task1/data sigmorphon2021
! wc -l sigmorphon2021/*/*.tsv

   4168 sigmorphon2021/high/eng_us_dev.tsv
   4168 sigmorphon2021/high/eng_us_test.tsv
  33344 sigmorphon2021/high/eng_us_train.tsv
    100 sigmorphon2021/low/ady_dev.tsv
    100 sigmorphon2021/low/ady_test.tsv
    800 sigmorphon2021/low/ady_train.tsv
    100 sigmorphon2021/low/gre_dev.tsv
    100 sigmorphon2021/low/gre_test.tsv
    800 sigmorphon2021/low/gre_train.tsv
    100 sigmorphon2021/low/ice_dev.tsv
    100 sigmorphon2021/low/ice_test.tsv
    800 sigmorphon2021/low/ice_train.tsv
    100 sigmorphon2021/low/ita_dev.tsv
    100 sigmorphon2021/low/ita_test.tsv
    800 sigmorphon2021/low/ita_train.tsv
    100 sigmorphon2021/low/khm_dev.tsv
    100 sigmorphon2021/low/khm_test.tsv
    800 sigmorphon2021/low/khm_train.tsv
    100 sigmorphon2021/low/lav_dev.tsv
    100 sigmorphon2021/low/lav_test.tsv
    800 sigmorphon2021/low/lav_train.tsv
    100 sigmorphon2021/low/mlt_latn_dev.tsv
    100 sigmorphon2021/low/mlt_latn_test.tsv
    800 sigmorphon2021/low/mlt_latn_train.tsv
    100 sigmo

Sigmorphon 2020 contains 3600 training items per language.

In [4]:
! git clone --depth 1 https://github.com/sigmorphon/2020.git

Cloning into '2020'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 258 (delta 56), reused 227 (delta 52), pack-reused 0
Receiving objects: 100% (258/258), 15.71 MiB | 13.40 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [5]:
! ln -fs 2020/task1/data sigmorphon2020
! wc -l sigmorphon2020/*/*.tsv

    450 sigmorphon2020/dev/ady_dev.tsv
    450 sigmorphon2020/dev/arm_dev.tsv
    450 sigmorphon2020/dev/bul_dev.tsv
    450 sigmorphon2020/dev/dut_dev.tsv
    450 sigmorphon2020/dev/fre_dev.tsv
    450 sigmorphon2020/dev/geo_dev.tsv
    450 sigmorphon2020/dev/gre_dev.tsv
    450 sigmorphon2020/dev/hin_dev.tsv
    450 sigmorphon2020/dev/hun_dev.tsv
    450 sigmorphon2020/dev/ice_dev.tsv
    450 sigmorphon2020/dev/jpn_dev.tsv
    450 sigmorphon2020/dev/kor_dev.tsv
    450 sigmorphon2020/dev/lit_dev.tsv
    450 sigmorphon2020/dev/rum_dev.tsv
    450 sigmorphon2020/dev/vie_dev.tsv
    450 sigmorphon2020/test/ady_test.tsv
    450 sigmorphon2020/test/arm_test.tsv
    450 sigmorphon2020/test/bul_test.tsv
    450 sigmorphon2020/test/dut_test.tsv
    450 sigmorphon2020/test/fre_test.tsv
    450 sigmorphon2020/test/geo_test.tsv
    450 sigmorphon2020/test/gre_test.tsv
    450 sigmorphon2020/test/hin_test.tsv
    450 sigmorphon2020/test/hun_test.tsv
    450 sigmorphon2020/test/ice_test.tsv
    4

## Command line options explained
### Data:
- **`--train TRAIN`**: Required path to the training dataset: `path/to/LANG_trains.tsv`. Required unless `--precomputed-train` is used along with `--vocabulary`.
- **`--dev DEV`**: Required path to the development (validation) dataset: `path/to/LANG_dev.tsv`
- **`--test TEST`**: Optional path to the test dataset: `path/to/LANG_test.tsv`
- **`--output OUTPUT`**: The output directory where the training results will be saved: `path/to/directory`. If output directory exists, the trainer stops.
- **`--precomputed-train PRECOMPUTED_TRAIN`**: Path to the precomputed training dataset. Must be accompanied by `--vocabulary`.
- **`--save-precomputed-train`**: If flag is set, stores the precomputed training dataset actions. Useful for speeding up subsequent training runs on large datasets. Create file `precomputed_train.pkl` in output directory.
- **`--vocabulary VOCABULARY`**: Path to the vocabulary file: `path/to/vocabulary.pkl` Required if `--precomputed-train` is provided.



### Model Configuration:
- **`--char-dim CHAR_DIM`**: The dimensionality of the character embedding vector. Defaults to 100.
- **`--feat-dim FEAT_DIM`**: The dimensionality of any additional feature embeddings, assuming UniMorph format data. Defaults to
- **`--action-dim ACTION_DIM`**: The dimensionality of the action embeddings.
- **`--enc-type {lstm,transformer}`**: The type of encoder to use, either LSTM or Transformer. Defaults to `lstm`.
- **`--enc-layers ENC_LAYERS`**: The number of layers in the encoder. Defaults 1.
- **`--enc-dropout ENC_DROPOUT`**: The internal layer dropout. Defaults 0.
- **`--dec-hidden-dim DEC_HIDDEN_DIM`**: The dimension of the hidden state in the LSTM decoder. Defaults to 200.
- **`--dec-layers DEC_LAYERS`**: The number of layers in the LSTM decoder. Defaults 1.

### Training and Evaluation:
- **`--pytorch-seed PYTORCH_SEED`**: Sets the random seed for PyTorch to ensure reproducibility of results. Random number is not set.
- **`--epochs EPOCHS`**: The maximum number of epochs to train. Defaults to 60.
- **`--batch-size BATCH_SIZE`**: The batch size for training. Defaults to 5.
- **`--eval-batch-size EVAL_BATCH_SIZE`**: The batch size for evaluation. Defaults to the same size as `--batch-size` if not specified.
- **`--patience PATIENCE`**: The number of epochs to wait for improvement before early stopping. Defaults to 12.
- **`--train-subset-eval-size TRAIN_SUBSET_EVAL_SIZE`**: The proportion of training data to use for evaluating training accuracy each epoch. Defaults to 5 (5% of the data)

### Optimization:
- **`--optimizer {adam,adamw,adadelta}`**: The optimizer to use during training. Defaults to `adadelta`.
- **`--scheduler {inv_sr,reduce_on_plateau}`**: The scheduler to use for adjusting the learning rate. Defaults to
- **`--loss-reduction {sum,mean}`**: The method for reducing the loss during training, either by summing or averaging. Defaults to `mean`.
- **`--grad-accumulation GRAD_ACCUMULATION`**: The number of steps over which gradients are accumulated before performing an optimization step.

### Decoding and Regularization:
- **`--beam-width BEAM_WIDTH`**: The width of the beam for beam search decoding. A value less than 1 disables beam search. Defaults to 4.
- **`--nfd`**: If flag is set, trains on NFD-normalized data and outputs results in NFC. Helps for highly composed script systems like Hangul.

### Device Configuration:
- **`--device DEVICE`**: Specifies the device to run the training on (e.g., CPU, GPU). Defaults to `cpu`. Set to `cuda` for GPU

### Specialized Training Options:
- **`--sed-em-iterations SED_EM_ITERATIONS`**: The number of Expectation-Maximization iterations for SED. Defaults to 10. Can be time-consuming.
- **`--sed-params SED_PARAMS`**: Path to learned SED parameters. `path/to/sed.pkl`


## Training a low-resource model

Computing a good SED model happens on CPU and takes some time. Therefore it's better to compute them once per data set and reuse it later. We use dummy training parameters to avoid long training.

In [8]:
! mkdir -p sed-2021/low_ita/
! trans-train --train sigmorphon2021/low/ita_train.tsv --dev sigmorphon2021/low/ita_dev.tsv --sed-em-iterations 5 --output /tmp/sed-ita-2021/low_ita \
      --epochs 1 --beam-width 0 --batch-size 200 --device cuda \
  && cp /tmp/sed-ita-2021/low_ita/sed.pkl sed-2021/low_ita && rm -r /tmp/sed-ita-2021/low_ita

INFO: pytorch_seed   : None
INFO: train          : sigmorphon2021/low/ita_train.tsv
INFO: precomputed_train: None
INFO: save_precomputed_train: False
INFO: vocabulary     : None
INFO: dev            : sigmorphon2021/low/ita_dev.tsv
INFO: test           : None
INFO: output         : /tmp/sed-ita-2021/low_ita
INFO: nfd            : False
INFO: char_dim       : 100
INFO: feat_dim       : None
INFO: action_dim     : 100
INFO: enc_type       : lstm
INFO: dec_hidden_dim : 200
INFO: dec_layers     : 1
INFO: beam_width     : 0
INFO: patience       : 12
INFO: epochs         : 1
INFO: batch_size     : 200
INFO: eval_batch_size: None
INFO: loss_reduction : mean
INFO: grad_accumulation: 1
INFO: train_subset_eval_size: 5
INFO: optimizer      : adadelta
INFO: scheduler      : None
INFO: sed_em_iterations: 5
INFO: sed_params     : None
INFO: device         : cuda
INFO: enc_hidden_dim : 200
INFO: enc_layers     : 1
INFO: enc_bidirectional: True
INFO: enc_dropout    : 0.0
INFO: lr             : 1.0
INF

In [9]:
! rm -rf result_ita_low_sed3_defaults  # remove earlier runs
! trans-train --train sigmorphon2021/low/ita_train.tsv --dev sigmorphon2021/low/ita_dev.tsv  --test sigmorphon2021/low/ita_test.tsv \
  --output result_ita_low_sed3_defaults \
  --enc-layers 2 --enc-dropout 0.1 \
  --batch-size 20 --beam-width 0 --epochs 100 --patience 20 --scheduler reduce_on_plateau --optimizer adam \
  --sed-params sed-2021/low_ita/sed.pkl --device cuda

INFO: pytorch_seed   : None
INFO: train          : sigmorphon2021/low/ita_train.tsv
INFO: precomputed_train: None
INFO: save_precomputed_train: False
INFO: vocabulary     : None
INFO: dev            : sigmorphon2021/low/ita_dev.tsv
INFO: test           : sigmorphon2021/low/ita_test.tsv
INFO: output         : result_ita_low_sed3_defaults
INFO: nfd            : False
INFO: char_dim       : 100
INFO: feat_dim       : None
INFO: action_dim     : 100
INFO: enc_type       : lstm
INFO: dec_hidden_dim : 200
INFO: dec_layers     : 1
INFO: beam_width     : 0
INFO: patience       : 20
INFO: epochs         : 100
INFO: batch_size     : 20
INFO: eval_batch_size: None
INFO: loss_reduction : mean
INFO: grad_accumulation: 1
INFO: train_subset_eval_size: 5
INFO: optimizer      : adam
INFO: scheduler      : reduce_on_plateau
INFO: sed_em_iterations: 10
INFO: sed_params     : sed-2021/low_ita/sed.pkl
INFO: device         : cuda
INFO: enc_hidden_dim : 200
INFO: enc_layers     : 2
INFO: enc_bidirectional: T

Inspect the output of the training and evaluation

In [31]:
! ls -lh result_ita_low_sed3_defaults/

total 7.8M
-rw-r--r-- 1 root root 7.8M Nov 20 22:11 best.model
-rw-r--r-- 1 root root  896 Nov 20 22:11 dev_greedy.eval
-rw-r--r-- 1 root root 2.3K Nov 20 22:11 dev_greedy.predictions
-rw-r--r-- 1 root root  897 Nov 20 22:11 test_greedy.eval
-rw-r--r-- 1 root root 2.2K Nov 20 22:11 test_greedy.predictions
-rw-r--r-- 1 root root  945 Nov 20 22:11 train.log
-rw-r--r-- 1 root root 1.1K Nov 20 22:10 vocabulary.pkl


### Compare the ground truth with the output for easy consumption

In [10]:
! diff -y --suppress-common result_ita_low_sed3_defaults/test_greedy.predictions 2021-task1/data/low/ita_test.tsv

avesse	a v ɛ s s e					      |	avesse	a v e s s e
biscotto	b i s k o t t o				      |	biscotto	b i s k ɔ t t o
cellula	t͡ʃ e l l u l a					      |	cellula	t͡ʃ ɛ l l u l a
cuoco	k u ɔ k o					      |	cuoco	k w ɔ k o
difficile	d i f f i t͡ʃ i l e			      |	difficile	d i f f i t ʃ i l e
dirompere	d i r ɔ m p e r e			      |	dirompere	d i r o m p e r e
effetto	e f f e t t o					      |	effetto	e f f ɛ t t o
episodio	e p i z o d j o				      |	episodio	e p i z ɔ d j o
linea	l i n ɛ a					      |	linea	l i n e a
maria	m a r j a					      |	maria	m a r i a
mio	m j o						      |	mio	m i o
nome	n ɔ m e						      |	nome	n o m e
ormai	ɔ r m a i					      |	ormai	o r m a i
ottenere	o t t ɛ n e r e				      |	ottenere	o t t e n e r e
ragazzo	r a ɡ a t t t s t s o				      |	ragazzo	r a ɡ a t t͡s o
rumeno	r u m e n o					      |	rumeno	r u m ɛ n o
so	s o						      |	so	s ɔ
stirpe	s t j r p e					      |	stirpe	s t i r p e
sì	s ì						      |	sì	s i
tempestivo	t e m p ɛ s t i v o			      |	tempest


Go to http://ipa-reader.xyz/ for pronouncing IPA phones in different languages (English, Italian, French, etc.) White-space must be removed.



Use the official evaluation script for the computation of WER and LER

In [11]:
! paste 2021-task1/data/low/ita_test.tsv result_ita_low_sed3_defaults/test_greedy.predictions | cut -f 2,4 > test.tsv
! python 2020/task1/evaluation/evaluate.py test.tsv



WER:	25.00
LER:	5.28
